In [1]:
#!pip install web3
#!pip install eth-tester


In [ ]:
from web3 import Web3, HTTPProvider, EthereumTesterProvider
from web3.middleware import geth_poa_middleware
from os.path import exists
from eth_account.messages import encode_defunct
from config.definitions import ROOT_DIR
from threading import Thread
from eth_abi import encode_abi, decode_abi


import json
import requests
import os
import asyncio
import time
import subprocess
import re 




nftOTT = "0xAf47c9D246fEF48C6AAc885353A86Cf06B8Ec4E5" #Address of the NFT contract
permissionedAddress = "0x56da4f2C76bbb7B56da0b287fC9d525872638145"
operator ="0xc9e93b4E813c6818975ea166B0CfEc001454aD0B" #Address of IBN
employeeRepo= "0x862052D18e7B8f5261E2601b94bE2197f58A86Ae"
policyRules="0xB6082Ff8C8765eE6AB4650f3AAAcBCF81876fEa7"

myOTT = None

abiFolder = os.path.join(ROOT_DIR, 'ABI')
with open(abiFolder+"/"+"ottNFT.json") as file:
    abiNFT = json.load(file)
    
with open(abiFolder+"/"+"accountRules.json") as file:
    abi = json.load(file)
with open(abiFolder+"/"+"repoList.json") as file:
    abiEmpRep = json.load(file)
with open(abiFolder+"/"+"policyRules.json") as file:
    abiPolicy = json.load(file)
    
rpcURL = "http://localhost:105/"
obj = {
    "username": "admin",
    "password": "admin"
}
authURL = "https://orchestrator.ncl.lab:1280/edge/management/v1/authenticate?method=password"

In [3]:
def createIdentity():
    ##print("Input the MFA verification message that you received in your E-mail")
    ##msg = str(input())
    #msg = "verified"
    
    #private_key = my_account.privateKey
    #message = encode_defunct(text=msg)
    #signed_message = w3.eth.account.sign_message(message, private_key=private_key)
    #signedMessage = signMessage(secretMessage, private_key) #Sign message to ensure proper identity
    jsonobj = {
        "signature": '0xefe4a65c4f8eabce4ed754ae59481ba0cad190270a916d1d00c00072e1b30796517b46f64d0d4a446a5f2718e31614ace012b8b87b82fd3e501c41583ee2a0ad1c',
        "address": '0xe542D6B2f22Fa8acA8de6bB1B65EC0a6028E5cA6'
    }


    print(jsonobj)

    createIdentity = requests.post(
    rpcURL+"/createIdentity/",
    verify=False,
    json = jsonobj
    )
    #print(createIdentity.text)

In [4]:
def safePasswordInput( my_encKey, num_retries = 3 ):
    for attempt_no in range(num_retries):
        try:
            passw = str(input())
            dec_key = w3.eth.account.decrypt(my_encKey, passw)
            return dec_key
        except ValueError as error:
            if attempt_no < (num_retries - 1):
                print("Error: Invalid password")
            else:
                raise error

In [5]:
def setApproval(address):
    tokensOwned = nftOTT_instance.functions.balanceOf(my_account._address).call() #Get the status of the account
    check_sum = w3.toChecksumAddress(my_account._address)
    print("Tokens Owned", tokensOwned)
    trans = nftOTT_instance.functions.setApprovalForAll(address, True).buildTransaction({"from": check_sum,"gasPrice": w3.eth.gas_price,"nonce": w3.eth.get_transaction_count(check_sum),"chainId": 2022}) #build RAW transaction supported by BESU
    signed_txn = w3.eth.account.sign_transaction(trans, my_account.privateKey) #Sign transaction using our own private key
    txn_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction) #Send transaction to BESU
    tx_receipt = w3.eth.wait_for_transaction_receipt(txn_hash.hex())  #Gets a receipt from the Blockchain
    return tx_receipt

In [6]:
def getBlockKey(file_exists):
    if not file_exists:
        print("Please provide an entropy phrase for your secured Blockchain Keys:")
        entropy = str(input())
        my_account = w3.eth.account.create(entropy)
        print("Please provide a password for encryptyion of Blockchain Keys:")
        passw = str(input())
        my_encAcc= w3.eth.account.encrypt(my_account.privateKey, passw)
        #print(my_encAcc)
        with open('encyptedKeyIBN.json', 'w') as json_file:
            json.dump(my_encAcc, json_file)
        return my_account
    else:
        with open('encyptedKeyIBN.json') as my_key:
            my_encKey = json.load(my_key)
        print(my_encKey)
        #print(type(my_encKey))
        print("Enter your password for decryption:")
        #passw = str(input())
        try:
            dec_key =safePasswordInput(my_encKey)  
            my_account = w3.eth.account.privateKeyToAccount(dec_key)
            return my_account
        except ValueError as error:
            print("Too many wrong passwords!!", error)
            raise error


In [7]:
"""def createIdentity():
 #   print("Input the MFA verification message that you received in your E-mail")
    msg = str(input())
    #msg = "verified"
    private_key = my_account.privateKey
    message = encode_defunct(text=msg)
    signed_message = w3.eth.account.sign_message(message, private_key=private_key)

    print(signed_message)

    createIdentity = requests.post(
    rpcURL+"/enroll/",
    verify=False,
    data = signed_message.signature
    )
    #print(createIdentity.text)
"""
    

'def createIdentity():\n #   print("Input the MFA verification message that you received in your E-mail")\n    msg = str(input())\n    #msg = "verified"\n    private_key = my_account.privateKey\n    message = encode_defunct(text=msg)\n    signed_message = w3.eth.account.sign_message(message, private_key=private_key)\n\n    print(signed_message)\n\n    createIdentity = requests.post(\n    rpcURL+"/enroll/",\n    verify=False,\n    data = signed_message.signature\n    )\n    #print(createIdentity.text)\n'

In [8]:
def callSomething(signedMessage):
    print(signedMessage)
    jsonobj = {
        "signedM": signedMessage
    }
    something = requests.post(
    rpcURL+"/decrypt/",
    verify=False,
    json = jsonobj
    )
    
    print(something.text)
    

In [9]:
def checkBalance():
    check_sum = w3.toChecksumAddress(my_account._address)
    balance = w3.eth.get_balance(check_sum)
    print(balance)

    

In [10]:
# define function to handle events and print to the console
def handle_event(event):
    eventS = Web3.toJSON(event)
    eventJSON = json.loads(eventS)
    print(eventJSON)
    tokenId = eventJSON["args"]["tokenId"]
    return tokenId
    # and whatever


# asynchronous defined function to loop
# this loop sets up an event filter and is looking for new entires for the "PairCreated" event
# this loop runs on a poll interval
#async def log_loop(event_filter, poll_interval):
def log_loop(event_filter, poll_interval):
    while True:
        for Transfer in event_filter.get_new_entries():
            #print("TEST")
            tokenId = handle_event(Transfer)
            global myOTT
            myOTT=tokenId
            print(tokenId)
        time.sleep(poll_interval)
        #await asyncio.sleep(poll_interval)

In [11]:
def eventFilter():
    event_filter = nftOTT_instance.events.Transfer.createFilter(fromBlock='latest', argument_filters={'to': my_account.address})
    print(event_filter)
    worker = Thread(target=log_loop, args=(event_filter, 2), daemon=True)
    worker.start()
    #block_filter = web3.eth.filter('latest')
    # tx_filter = web3.eth.filter('pending')
    #loop = asyncio.get_event_loop()
    #print("LOOP", loop)
    #try:
       # loop.run_until_complete(
          #  asyncio.gather(
            #    log_loop(event_filter, 2)))
                # log_loop(block_filter, 2),
                # log_loop(tx_filter, 2)))
    #finally:
        # close loop to free up system resources
      #  loop.close()


In [12]:
def enroll(tokenId):
    tokenURI = nftOTT_instance.functions.tokenURI(tokenId).call() #Get the OTT information
    with open('ott.jwt', 'w') as f:
        f.write(tokenURI)
    output = subprocess.run(["ziti.exe", "edge", "enroll", "-j", "ott.jwt", "-o", "myId.json"], capture_output=True, text=True, check=True)
    print(output)
    



    

In [13]:
def signMessage(message, privKey):
    singlequote ="'"
    doublequote = '"'
    cmd = 'node -e \"require(\'./signMe.js\').signMessage(\'{}\',\'{}\')\"' #If fail, check quotes
    pattern = r'\\n(.*)\\n'
    
 
    privKey = w3.toHex(privKey)
    output = subprocess.check_output(cmd.format(message, privKey), shell=True)
    
    
    #signed_message = w3.eth.sign(privKey,text=message)
    print(cmd.format(message,privKey))
    substring = re.search(pattern, str(output)).group(1)
    print(substring)
    return substring
    

In [14]:
def mintOTTNFT(address):
    check_sum = w3.toChecksumAddress(address)
    ott = "PENICHI"  
    #mintOTTNFT(ott, address)
    totalSupply = nftOTT_instance.functions.totalSupply().call() #Get the total amount of tokens created
    check_sum = w3.toChecksumAddress(my_account._address)
    print("Totalsupply", totalSupply, address, ott)
    tokenId = totalSupply+1
    print(tokenId)
    trans = nftOTT_instance.functions.mint(address,tokenId,ott).buildTransaction({"from": check_sum,"gasPrice": w3.eth.gas_price,"nonce": w3.eth.get_transaction_count(check_sum),"chainId": 2022}) #build RAW transaction supported by BESU
    signed_txn = w3.eth.account.sign_transaction(trans, my_account.privateKey) #Sign transaction using our own private key
    print(signed_txn)
    txn_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction) #Send transaction to BESU

In [15]:
def getIdentityInfoN(identity):
    authResponse = requests.post(authURL, json=obj, verify=False,)
    jsonResponse = json.loads(authResponse.text)
    #print(authResponse.text)

    identityInfo = requests.get(
    f"https://172.18.102.169:1280/edge/management/v1/identities?filter=(name contains \"{identity}\")",
    verify=False,
    headers={"zt-session": jsonResponse['data']['token']}
        )
    identityResponse = json.loads(identityInfo.text)
    ##auth = identityResponse["data"][0]["authenticators"]

    print(identityResponse["data"])
    ##print(len(auth))


In [16]:
if __name__ == '__main__':
    w3 = Web3(HTTPProvider('http://172.18.102.169:9545')) #If access to our Local lockchain
    w3.isConnected()
    #w3 = Web3(EthereumTesterProvider()) #If no internet connectivity
    w3.middleware_onion.inject(geth_poa_middleware, layer=0) #For compatibility with POA consensus chains
    print ("Latest Ethereum block number" , w3.eth.block_number)
    nftOTT_instance = w3.eth.contract(address = nftOTT, abi = abiNFT) #Creates a contract instance for the OTT-NFT
    verify_instance = w3.eth.contract(address = employeeRepo, abi = abiEmpRep) #Creates a contract instance for the employee Repo 

    
    file_exists = exists("encyptedKey.json")
    #print(file_exists)
    try:
        my_account = getBlockKey(file_exists)
        try:
            print(my_account.address)
        except NameError as error:
            print("There is no Ethereum account!!", error)
        latest_block = w3.eth.get_block('latest')
        print(latest_block)
    except:
        print("Verify your Ethereum Account")

KeyboardInterrupt: 

In [ ]:
checkBalance() #Checks balance of current account

In [ ]:
eventFilter()

In [ ]:

createIdentity() #Creates a new Identity

In [ ]:
receipt = setApproval(operator)  #Approves the use of the NFT to the IBN blockchain account
print("Approved", receipt)

In [ ]:
print(myOTT)
enroll(myOTT)

In [ ]:
signMessage('verified', my_account.privateKey)

In [ ]:
callSomething('0x695878821aa62bd098ec606b5a7a31b0305de019bef4e6118266fa66d7b833b31805623374e15c45cb3f8aa9ba86ecb0df63f30142d3c57d44087765bae2a0791b')

In [ ]:
createIdentity()

In [ ]:
def verifyMFA():
   
    myhex =  Web3.toBytes(hexstr='0xefe4a65c4f8eabce4ed754ae59481ba0cad190270a916d1d00c00072e1b30796517b46f64d0d4a446a5f2718e31614ace012b8b87b82fd3e501c41583ee2a0ad1c')
    print(myhex)
    encoded = encode_abi(['bytes', 'address'], [myhex, '0xe542D6B2f22Fa8acA8de6bB1B65EC0a6028E5cA6'])
    encodedhex = w3.toHex(encoded)
    print(encoded, encodedhex)
    
    decoded = decode_abi(['bytes', 'address'], b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe5B\xd6\xb2\xf2/\xa8\xac\xa8\xdek\xb1\xb6^\xc0\xa6\x02\x8e\\\xa6\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00A\xef\xe4\xa6\\O\x8e\xab\xceN\xd7T\xaeYH\x1b\xa0\xca\xd1\x90'\n\x91m\x1d\x00\xc0\x00r\xe1\xb3\x07\x96Q{F\xf6M\rJDj_'\x18\xe3\x16\x14\xac\xe0\x12\xb8\xb8{\x82\xfd>P\x1cAX>\xe2\xa0\xad\x1c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00")
    print(decoded)
    resultMFA = verify_instance.functions.verifyMFA2('0xe542D6B2f22Fa8acA8de6bB1B65EC0a6028E5cA6', 0x1c, '0xefe4a65c4f8eabce4ed754ae59481ba0cad190270a916d1d00c00072e1b30796', '0x517b46f64d0d4a446a5f2718e31614ace012b8b87b82fd3e501c41583ee2a0ad').call({'from': "0xc9e93b4E813c6818975ea166B0CfEc001454aD0B"}) #Get the status of the account
    struct =  verify_instance.functions.getEmployeeByEmployeeId(5678).call({'from': "0xc9e93b4E813c6818975ea166B0CfEc001454aD0B"})
    print(struct)
    print(resultMFA)
    return resultMFA

In [ ]:
verifyMFA()

In [ ]:
w3.clientVersion

In [ ]:
mintOTTNFT("0xe542D6B2f22Fa8acA8de6bB1B65EC0a6028E5cA6")

In [ ]:
getIdentityInfoN("0xe542D6B2f22Fa8acA8de6bB1B65EC0a6028E5cAd6")

In [17]:
hashID = Web3.keccak(text="{'id': 'htMtcA1rzK', 'name': '0xe542D6B2f22Fa8acA8de6bB1B65EC0a6028E5cA6', 'createdAt': '2022-08-09T02:20:51.839Z', 'updatedAt': '2022-08-09T02:20:51.839Z', 'auth': {'cert': {'fingerprint': 'ef94b3c2a37cda57341177bc4b00f1b43e3a0e73', 'id': 'KkmicA1kz'}}}")

print(str(hashID))
pinichi = Web3.toHex(hashID)
print(type(pinichi))


b'\xf5\xd1\x01*s\n.0\x99wRL\xfe\xdf\xc1\xda\x1dA}\xe42Wb%j\x0f\xb3\x14\xb4\xc8\xd3&'
<class 'str'>


In [ ]:
def getIdentityInfo():
    authResponse = requests.post(authURL, json=obj, verify=False,)
    jsonResponse = json.loads(authResponse.text)
    #print(authResponse.text)

    identityInfo = requests.get(
    f"https://172.18.102.169:1280/edge/management/v1/identities/3gj81K1r4K",
    verify=False,
    headers={"zt-session": jsonResponse['data']['token']}
        )
    identityResponse = json.loads(identityInfo.text)
    auth = identityResponse["data"]["authenticators"]
    print(identityResponse["data"]["authenticators"])
    print(len(auth))

In [18]:
getIdentityInfo()

NameError: name 'getIdentityInfo' is not defined

In [47]:
from collections import namedtuple
w3 = Web3(HTTPProvider('http://127.0.0.1:8545')) #If access to our Local lockchain

class Policy():
    "Stores name and place pairs"
    def __init__(self, policyId, policyHash):
        self.policyId = policyId
        self.policyHash = policyHash
        
w3.middleware_onion.inject(geth_poa_middleware, layer=0) #For compatibility with POA consensus chains
nonce = 0 #Need to find a better way for nonce tracking
policyRules_instance = w3.eth.contract(address = policyRules, abi = abiPolicy)
roles = [1, 4, 5, 6]
metadataList = []
Policy = namedtuple("Policy",["policyId","hash"])




In [48]:
def getPolicies():
    policies = policyRules_instance.functions.getAllPolicies().call() #Get all the policies
    if len(policies) == 0:
        return 0
    else:
        return policies

In [49]:
def detect(list_a, list_b):
    result = set(list_a) & set(list_b)
    print(list(result))
    
    return result

In [50]:
policyIds = []
activePolicies = getPolicies()
for p in activePolicies:
    print("This Policy", p)
    a = detect(roles,p[1])
    print(a)
    if a: 
        policyIds.append(Policy(p[0],p[4])) 
    print("RESULT", a)
print("List of policies", policyIds)


    
    

This Policy (1, [1], 1, '0xB6082Ff8C8765eE6AB4650f3AAAcBCF81876fEa7', 'TEST')
[1]
{1}
RESULT {1}
This Policy (2, [1, 2], 2, '0xB6082Ff8C8765eE6AB4650f3AAAcBCF81876fEa7', 'TEST')
[1]
{1}
RESULT {1}
This Policy (3, [3], 2, '0xB6082Ff8C8765eE6AB4650f3AAAcBCF81876fEa7', 'TEST')
[]
set()
RESULT set()
List of policies [Policy(policyId=1, hash='TEST'), Policy(policyId=2, hash='TEST')]


In [76]:
def getActivePolicies(policies):
    activePolicyList = []
    for p in policies:
        print("This Policy", p)
        a = detect(roles,p[1])
        if a: 
            activePolicyList.append(Policy(p[0],p[4])) 
        print("RESULT", list(a))
    print("List of policies", activePolicyList)
    result = json.dumps(activePolicyList[0]._asdict())##Important for the metadata
    print("JSON", result)
    return activePolicyList

In [77]:
thePolicies = getPolicies()  ##NEED TO QUERY THE TOTAL POLICIES


In [78]:
metadataList = getActivePolicies(thePolicies) ##MAP roleList with the Total policies.


This Policy (1, [1], 1, '0xB6082Ff8C8765eE6AB4650f3AAAcBCF81876fEa7', 'TEST')
[1]
RESULT [1]
This Policy (2, [1, 2], 2, '0xB6082Ff8C8765eE6AB4650f3AAAcBCF81876fEa7', 'TEST')
[1]
RESULT [1]
This Policy (3, [3], 2, '0xB6082Ff8C8765eE6AB4650f3AAAcBCF81876fEa7', 'TEST')
[]
RESULT []
List of policies [Policy(policyId=1, hash='TEST'), Policy(policyId=2, hash='TEST')]
JSON {"policyId": 1, "hash": "TEST"}
